In [1]:
import sys

from PyMPDATA_examples.Arabas_and_Farhat_2020.options import OPTIONS

if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PyMPDATA-examples')

In [38]:
import os
import numpy as np
from scipy.interpolate import make_interp_spline
from matplotlib import pyplot, colors, colormaps, patches
from open_atmos_jupyter_utils import show_plot
# from PyMPDATA_examples.Magnuszewski_et_al_2025.common import OPTIONS
from PyMPDATA import Options

from PyMPDATA_examples.Magnuszewski_et_al_2025.asian_option import AsianArithmetic, Settings
from PyMPDATA_examples.Magnuszewski_et_al_2025.monte_carlo import BSModel, FixedStrikeArithmeticAsianOption
from PyMPDATA_examples.utils.financial_formulae import Black_Scholes_1973
import PyMPDATA_examples.utils.financial_formulae.asian_option as asian_analytic
from tqdm import tqdm
from PyMPDATA_examples.Magnuszewski_et_al_2025.monte_carlo import BSModel, FixedStrikeArithmeticAsianOption

In [55]:
# base_nx = 21
# base_ny = 31

mc_n_paths = 100000
mc_seed = 42
mc_path_points = 1000


SETTINGS = Settings(
    T=1,
    K=100,
    r=.08,
    sgma=.4,
    S_max=200,
    S_min=50,
)
variant = "call"

OPTIONS = Options(
        n_iters=2,
        nonoscillatory=True,
        non_zero_mu_coeff=True,
    )


In [56]:

# scale=4
spot=100
# resolution2 = {k:v*scale for k,v in resolution.items()}

In [58]:
mc_model = BSModel(
        T=1,
        sigma=0.4,
        r=0.08,
        M=mc_path_points,
        S0=spot,
        seed=mc_seed
    )
arithmetic_option = FixedStrikeArithmeticAsianOption(1, 100, variant, mc_model, mc_n_paths)
price_by_mc = arithmetic_option.price_by_mc()

In [59]:
price_by_mc

10.643044762553435

In [67]:
resolution = {
    'nt': 2000,
    'nx': 11,
    'ny': 16,
}

In [70]:
scales = [1, 2, 4, 8, 16]
results = {}
for scale in scales:
    resolution_scaled = {k: v * scale for k, v in resolution.items()}
    simulation = AsianArithmetic(SETTINGS, **resolution_scaled, options=OPTIONS, variant=variant)
    simulation.step(simulation.nt)
    simulation_price = simulation.solver.advectee.get()[:, 0]
    results[scale] = np.interp(spot, simulation.S, simulation_price)
    


courant_number_x=5.505889303431891e-20
cfl_condition=0.007499999999999996
courant_number_x=5.505889303431891e-20
cfl_condition=0.0077499999999999965
courant_number_x=5.505889303431891e-20
cfl_condition=0.007874999999999997
courant_number_x=5.505889303431891e-20
cfl_condition=0.007937499999999995
courant_number_x=5.505889303431891e-20
cfl_condition=0.007968749999999997


In [71]:
results

{1: 14.125965307980703,
 2: 12.042222803433566,
 4: nan,
 8: 10.926467884458024,
 16: nan}